In [1]:
import KGInt as kg
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
from ipyparallel import Client

%matplotlib inline

In [15]:
f = h5.File("Levy.h5", "r")

limits = f["limits"]["999"].value
alphas = np.linspace(f["alphas"]["min_alpha"].value,f["alphas"]["max_alpha"].value, f["alphas"]["length"].value)
dr     = f["dr"]["999"].value

l=kg.Levyd(1., alphas[999], limits[0], limits[1], dr, f["Lr"]["999"].value, 1e-8)


In [16]:
c3 = kg.Coulomb3d(l)
c2 = kg.Coulomb2d(l)

In [24]:
c2.set_k(15)

In [25]:
c3.set_k(15,15,15)

In [26]:
c3.integrate(100000)

1.2253053397709106

In [27]:
t = c2.integrate(1000000)

In [28]:
t**3

1.154861649051774